In [ ]:
import time
import numpy as np
import pandas as pd
from sklearn import metrics
from tabulate import tabulate
from datetime import timedelta
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_regression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier as sklearn_DecisionTreeClassifier

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import UnivariateFeatureSelector
from pyspark.ml.feature import ChiSqSelector, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import DecisionTreeClassifier as pyspark_DecisionTreeClassifier

In [ ]:
# Creating a new spark session
spark = SparkSession.builder.master("local[*]").appName("MLlib lab").getOrCreate()

In [ ]:
sparkDF = spark.read.option("maxColumns", 22285).csv("/mnt/the-data-transformers/toxicity.csv", header=True)
pandasDF = sparkDF.toPandas()

In [ ]:
len(sparkDF.columns)

Out[10]: 1025

In [ ]:
sparkDF.count()

Out[11]: 8992

In [ ]:
y = pandasDF.iloc[:, 1024]
X = pandasDF.iloc[:, 0:1023]

In [ ]:
'''
Function: sklearn_mutual_info_regression
INPUT:
------
X: Features
y: labels
'''
def sklearn_mutual_info_regression(X, y):
  # configure to select all features
  selector = SelectKBest(score_func=mutual_info_regression, k=400)
  # learn relationship from training data
  selector.fit(X, y)
  # transform train input data
  X_train_fs = selector.transform(X)
  
  X_new = X[X.columns[selector.get_support()]]
  
  train, test, train_labels, test_labels = train_test_split(X_new, y, test_size=0.30, random_state=42)
  
  clf = sklearn_DecisionTreeClassifier()
  clf = clf.fit(train, train_labels)

In [ ]:
sklearn_mutual_info_regression(X, y)